<a href="https://colab.research.google.com/github/viniciusrpb/natural_language_processing/blob/main/tweet_sentiment_classification_rnn_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.text.all import *
import pandas as pd

In [93]:
df = pd.read_csv('/content/SemEval2014-task3-train-valid.txt',sep='\t')
df.head()

,label,text,is_valid
0,negative,Theo Walcott is still shitc watch Rafa and Johnny deal with him on Saturday.,False
1,negative,its not that Im a GSP fanc i just hate Nick Diaz. cant wait for february.,False
2,negative,Iranian general says Israels Iron Dome cant deal with their missiles (keep talking like that and we may end up finding out),False
3,neutral,Tehranc Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,False
4,neutral,I sat through this whole movie just for Harry and Ron at christmas. ohlawd,False


In [105]:
dls_lm = DataBlock(blocks=(TextBlock.from_df('text', seq_len=8), CategoryBlock),
                      get_x=ColReader('text'),
                      get_y=ColReader('label'),
                      splitter=ColSplitter())

In [106]:
dls_lm = dls_lm.dataloaders(df, bs=64)
dls_lm.show_batch()

#dls = TextDataLoaders.from_df(df, path=path, text_col='text', label_col='label', valid_col='is_valid')
#dls.show_batch(max_n=3)

,text,category
0,xxbos # # $ $ # # xxmaj black xxmaj friday xxmaj deals xxmaj olympus xxup om - d xxup e - m5 xxmaj digital xxmaj camera - xxmaj black - with xxmaj olympus 12 - 50 mm f / xxunk - 5.6 xxup xxunk xxmaj zoom xxmaj lens - b … http : / / t.co / xxunk,neutral
1,xxbos xxmaj check it out ! ! xxmaj big xxmaj time xxmaj rush : xxmaj dance xxmaj party & xxmaj big xxmaj time xxmaj rush : xxmaj backstage xxmaj pass xxmaj pushed xxmaj back a xxmaj weekc xxmaj out xxmaj on xxmaj november 13 … http : / / t.co / xxunk,neutral
2,xxbos xxmaj make xxmaj sure xxmaj to xxmaj come xxmaj to xxmaj the xxmaj bob xxmaj jones xxmaj game xxmaj friday ! xxmaj free xxmaj hot xxmaj dogsc xxmaj xxunk & xxmaj food outside the gate ! & watch xxmaj bob xxmaj jones take on xxmaj austin xxmaj high !,neutral
3,xxbos # # $ $ # # xxmaj black xxmaj friday xxmaj deals xxmaj olympus xxup om - d xxup e - m5 xxunk xxmaj live xxup mos xxmaj xxunk xxmaj lens xxmaj camera with xxunk xxmaj xxunk xxup xxunk xxmaj xxunk … http : / / t.co / xxunk,neutral
4,xxbos xxmaj wellc its xxmaj monday again folks : - / xxmaj start of week 4 at xxup ou ! xxmaj butc its a new week in xxmaj xxunk & xxmaj port xxmaj charles ! xxmaj woo ! # xxup days # xxup gh ! xxmaj morning all ! :),positive
5,xxbos 4 xxmaj xxunk xxmaj hoops xxmaj xxunk tonight 1 . xxmaj san xxmaj diego xxmaj state @ xxmaj new xxmaj mexico 2 . xxmaj xxunk @ xxmaj west xxmaj virginia 3 . xxmaj st . xxmaj joe @ xxmaj xxunk 4 . xxmaj wichita xxmaj st @ xxup uni,neutral
6,"xxbos { haha z xxrep 3 i n xxrep 3 g } xxup rt xxunk xxrep 3 0 : xxmaj all across the xxup us , with xxmaj live xxmaj feeds down , xxmaj big xxmaj brother fans are seeing the xxmaj sun for the first time in months .",neutral
7,xxbos http : / / t.co / xxunk : xxmaj check xxmaj out the xxmaj box xxmaj art for xxmaj tomb xxmaj raider : xxmaj what do we think ? xxmaj the new xxmaj tomb xxmaj raider is out xxmaj march 5 … http : / / t.co / xxunk,neutral
8,xxbos xxmaj lots of rumors out there . xxmaj oilers are said to want xxmaj xxunk / xxmaj adam / 3rdc xxmaj sekera / xxmaj xxunk / 2ndc xxmaj sekera / xxmaj xxunk xxmaj xxunk / xxmaj adam for xxmaj sam xxmaj gagner . xxmaj who knows ? ?,neutral


In [107]:
print(dls_lm.vocab)

[['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', 'the', '.', '/', 'to', ':', '!', 'i', 'in', 'on', 'a', 'and', '#', 'of', 'for', '…', 'http', 't.co', 'is', 'you', 'at', 'tomorrow', 'with', 'it', '\\', '?', 'be', '-', 'my', 'that', 'may', 'have', ',', 'this', 'night', '3', 'will', 'day', 'just', 'going', 'u002c', 'm', 'nt', '&', 'was', 'we', 'out', 'all', 'but', 'me', 'see', 'he', 'are', 'game', 'so', '(', ')', 'from', 'tonight', 'do', 'if', 'saturday', 'time', 'up', 'get', '1st', 'friday', 'sunday', 'like', 'not', 'today', 'go', 'about', '..', 'its', 'as', 'can', 'one', 'good', 'rt', '4', 'new', "'s", '2', '"', 'now', 'by', 's', '2nd', 'what', 'they', 'last', 'monday', 'your', 'or', 'got', 'there', 'come', ':)', 'u2019s', 'his', 'has', 'love', 'thursday', 'no', 'back', 'when', 'u2019', 'then', 'watch', 'who', 'after', 'want', 'an', 'make', 'think', 'd', 'know', '3rd', 'ca', 'off', 'still', 'wait', 'show', '1', 'us', 'some', 'best', 'first', 'morning', 

### Cria um classificador de textos

/bin/bash: -c: line 0: syntax error near unexpected token `path/'train''
/bin/bash: -c: line 0: `ls(path/'train')'


In [108]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [110]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [113]:
learn = Learner(dls_lm, LMModel2(len(dls_lm.vocab), 64), loss_func=F.cross_entropy,metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

IndexError: ignored